**Ho va ten: Do Khanh Dang Khoa**

**MSSV: 19110348**

In [ ]:
!nvidia-smi

Sun Nov 27 02:37:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv

--2022-11-27 02:40:56--  https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/plain]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.004s  

2022-11-27 02:40:56 (118 MB/s) - ‘spam.csv’ saved [503663/503663]



In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 36.5 MB/s 


In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import regex as re
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, XLNetTokenizer

In [ ]:
data = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")
data.drop(["Unnamed: 2", "Unnamed: 3","Unnamed: 4"], axis=1, inplace=True)

In [ ]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()

In [ ]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...,0,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,0,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data["gt"], test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
sentences=data['text']
labels=data['gt']
len(sentences),len(labels)

(5572, 5572)

**1. Chạy mô hình bert-base-cased**

In [ ]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME = 'bert-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

In [ ]:
def Tokenizer(sentences):
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case = False, add_special_tokens=True,
                                                max_length=MAX_LENGTH, pad_to_max_length=True, use_fast=True)
    input_ids=[]
    attention_masks=[]

    for sent in tqdm(sentences):
        bert_inp=tokenizer.encode_plus(sent, add_special_tokens = True ,max_length=MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    return input_ids, attention_masks

In [ ]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 8262.00it/s]


In [ ]:
def create_model(model, config):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(model, config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [ ]:
model = create_model(model= MODEL_NAME, config=MODEL_CONFIG)

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [ ]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
                                        monitor="val_accuracy",
                                        verbos=1,
                                        save_best_only=True,
                                        mode="max")
history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=64, 
                    epochs=10,
                    callbacks=[mc],
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/10


53/53 [==============================] - 112s 2s/step - loss: 0.3868 - accuracy: 0.8468 - val_loss: 0.0941 - val_accuracy: 0.9776
Epoch 2/10
53/53 [==============================] - 94s 2s/step - loss: 0.1023 - accuracy: 0.9692 - val_loss: 0.0502 - val_accuracy: 0.9857
Epoch 3/10
53/53 [==============================] - 94s 2s/step - loss: 0.0572 - accuracy: 0.9838 - val_loss: 0.0451 - val_accuracy: 0.9874
Epoch 4/10
53/53 [==============================] - 94s 2s/step - loss: 0.0507 - accuracy: 0.9874 - val_loss: 0.0454 - val_accuracy: 0.9901
Epoch 5/10
53/53 [==============================] - 88s 2s/step - loss: 0.0347 - accuracy: 0.9934 - val_loss: 0.0525 - val_accuracy: 0.9892
Epoch 6/10
53/53 [==============================] - 88s 2s/step - loss: 0.0253 - accuracy: 0.9937 - val_loss: 0.0635 - val_accuracy: 0.9874
Epoch 7/10
53/53 [==============================] - 88s 2s/step - loss: 0.0174 - accuracy: 0.9964 - val_loss: 0.0673 - val_accuracy: 0.9874
Epoch 8/10
53/53 [============

In [ ]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 10s 283ms/step - loss: 0.0346 - accuracy: 0.9946


[0.034637127071619034, 0.994618833065033]

**2. Thay Bert bằng xlnet-base-cased**

In [ ]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME_2 = 'xlnet-base-cased'
MODEL_CONFIG_2 = XLNetTokenizer.from_pretrained(MODEL_NAME_2)

In [ ]:
def Tokenizer_2(sentences):
    tokenizer_2 = XLNetTokenizer.from_pretrained(MODEL_NAME_2, do_lower_case= False, add_special_tokens=True, max_length=MAX_LENGTH, pad_to_max_length=True)
    input_ids_2=[]
    attention_masks_2=[]

    for sent in tqdm(sentences):
        bert_inp=tokenizer_2.encode_plus(sent, add_special_tokens = True ,max_length=MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids_2.append(bert_inp['input_ids'])
        attention_masks_2.append(bert_inp['attention_mask'])

    input_ids_2=np.asarray(input_ids_2)
    attention_masks_2=np.array(attention_masks_2)
    return input_ids_2, attention_masks_2

In [ ]:
input_ids_train_xlnet, attention_masks_train_xlnet = Tokenizer_2(X_train)
input_ids_test_xlnet, attention_masks_test_xlnet = Tokenizer_2(X_test)
input_ids_valid_xlnet, attention_masks_vaild_xlnet = Tokenizer_2(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 3836.08it/s]


In [ ]:
model_2 = create_model(model = MODEL_NAME_2, config = MODEL_CONFIG_2)

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [ ]:
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model (TFXLNetModel)  TFXLNetModelOutput(  116718336   ['input_token[0][0]',            
                                last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                            

In [ ]:
model_2.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              metrics=["accuracy"])

In [ ]:
history_2 = model_2.fit([input_ids_train_xlnet, attention_masks_train_xlnet], 
                    y_train,
                    batch_size=16, 
                    epochs=4,
                    validation_data=([input_ids_valid_xlnet, attention_masks_vaild_xlnet], y_valid)
                    )

Epoch 1/4


209/209 [==============================] - 132s 563ms/step - loss: 0.0000e+00 - accuracy: 0.8220 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 2/4
209/209 [==============================] - 117s 561ms/step - loss: 0.0000e+00 - accuracy: 0.8333 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 3/4
209/209 [==============================] - 119s 569ms/step - loss: 0.0000e+00 - accuracy: 0.8327 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 4/4
209/209 [==============================] - 119s 570ms/step - loss: 0.0000e+00 - accuracy: 0.8447 - val_loss: 0.0000e+00 - val_accuracy: 0.8691


In [ ]:
model_2.evaluate([input_ids_test_xlnet, attention_masks_test_xlnet], y_test)

35/35 [==============================] - 12s 335ms/step - loss: 0.0000e+00 - accuracy: 0.8511


[0.0, 0.8511210680007935]

**3.Từ mô hình xlnet-base-cased ở những layers cuối hãy thay thành các FC layers có số units lần lượt là 50, 30, 10**

In [ ]:
def create_model_BT3(model, config):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(model, config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(50, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(30, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [ ]:
model_3 = create_model_BT3(model = MODEL_NAME_2, config = MODEL_CONFIG_2)

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [ ]:
model_3.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model_1 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_token[0][0]',            
 )                              last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                          

In [ ]:
model_3.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3),
              metrics=["accuracy"])

In [ ]:
history_3 = model_3.fit([input_ids_train_xlnet, attention_masks_train_xlnet], 
                    y_train,
                    batch_size=16, 
                    epochs=4,
                    validation_data=([input_ids_valid_xlnet, attention_masks_vaild_xlnet], y_valid)
                    )

Epoch 1/4


209/209 [==============================] - 132s 550ms/step - loss: 0.0000e+00 - accuracy: 0.8594 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 2/4
209/209 [==============================] - 116s 554ms/step - loss: 0.0000e+00 - accuracy: 0.8698 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 3/4
209/209 [==============================] - 117s 558ms/step - loss: 0.0000e+00 - accuracy: 0.8698 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 4/4
209/209 [==============================] - 118s 564ms/step - loss: 0.0000e+00 - accuracy: 0.8698 - val_loss: 0.0000e+00 - val_accuracy: 0.8691


In [ ]:
model_3.evaluate([input_ids_test_xlnet, attention_masks_test_xlnet], y_test)

35/35 [==============================] - 11s 321ms/step - loss: 0.0000e+00 - accuracy: 0.8511


[0.0, 0.8511210680007935]